<a href="https://colab.research.google.com/github/sunfflur/frequency-learning/blob/master/Exp_02_ViT-b/16/32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Experiment - ViTs

`Author:` [sunfflur](https://github.com/sunfflur)

> This experiment considers the 28 classes from Kylberg Texture Dataset being classified by ViT-b/32 and ViT-b/16 through transfer learning.


---

#### Setup

In [ ]:
pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 10.2 MB/s eta 0:00:00


In [ ]:
!pip install vit-keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=65d746c1cf492659d3205535d80f0a5f7dfd5c9b0b82383d8baf4f112420d8fc
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from vit_keras import vit, utils
from tensorflow.keras.layers import Layer, Dense, Conv1D, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
import seaborn as sn

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


#### Drive mount

In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 122536 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


#### Prepare the data

##### Kylberg Texture Dataset

In [ ]:
num_classes = 28
input_shape = (384, 384, 1)

### data loading ### 
path_train = '/content/drive/MyDrive/Mestrado/Datasets/kylberg-dataset/kylberg_train.npy'
path_ytrain = '/content/drive/MyDrive/Mestrado/Datasets/kylberg-dataset/kylberg_y_train.npy'
path_test = '/content/drive/MyDrive/Mestrado/Datasets/kylberg-dataset/kylberg_test.npy'
path_ytest = '/content/drive/MyDrive/Mestrado/Datasets/kylberg-dataset/kylberg_y_test.npy'

x_train, x_test = np.load(path_train), np.load(path_test)
y_train, y_test = np.load(path_ytrain), np.load(path_ytest)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

# adjust dimension and change from array to tensor
new_shape = (384,384)
x_train, x_test = np.asarray(tf.image.resize(x_train, new_shape)), np.asarray(tf.image.resize(x_test, new_shape))
y_train, y_test = y_train.reshape(y_train.shape[0], 1), y_test.reshape(y_test.shape[0], 1)
y_train, y_test = tf.stack(to_categorical(y_train)), tf.stack(to_categorical(y_test))

In [ ]:
def ds_split(ds, ds_size, train_split=0.9, val_split=0.1, shuffle=False, shuffle_size=None):
    assert (train_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=11)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    #test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds

In [ ]:
dstrain = tf.data.Dataset.from_tensor_slices((x_train,y_train))
dstest = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [ ]:
x_train_x, x_val_x = ds_split(dstrain, 3360)

In [ ]:
bs = 4
x_train1 = x_train_x.batch(bs, drop_remainder=True).map(lambda x,y: (tf.image.grayscale_to_rgb(x), y)).prefetch(tf.data.AUTOTUNE).cache() #
x_val1 = x_val_x.batch(bs, drop_remainder=True).map(lambda x,y: (tf.image.grayscale_to_rgb(x), y)).prefetch(tf.data.AUTOTUNE).cache()
x_test1 = dstest.batch(bs, drop_remainder=True).map(lambda x,y: (tf.image.grayscale_to_rgb(x), y)).cache().prefetch(tf.data.AUTOTUNE) 

#### Transfer learning - VITb16 - 384x384

In [ ]:
def pre_trained_model(input_shape, n_classes, optimizer, fine_tune=0):
    """
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. 
               0 = all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    classes = utils.get_imagenet_classes()
    vit_model = vit.vit_b16(
        image_size=input_shape[0],
        activation='sigmoid',
        pretrained=True,
        include_top=True,
        pretrained_top=True)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    
    if fine_tune > 0:
        for layer in vit_model.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in vit_model.layers:
            layer.trainable = False
    '''
    # Turn all the layers trainable.

    for layer in densenet_model.layers:
      layer.trainable = True
    '''
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = vit_model.layers[-2].output
    #top_model = Flatten(name="flatten")(top_model)
    #top_model = Dense(4096, activation='relu')(top_model)
    #top_model = Dense(128, activation='relu')(top_model)
    #top_model = Dropout(0.25)(top_model)
    output_layer = Dense(n_classes, activation='softmax', name='my_predictions')(top_model)
    
    # Group the convolutional base and new output into a Model object.
    model = Model(inputs=vit_model.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
tf.random.set_seed(11) #s

### inverse time decay  ###
inversetime_decay = tf.keras.optimizers.schedules.InverseTimeDecay(
  initial_learning_rate = 0.001, #0.01 - opt2 lr=0.001
  decay_steps = 3360/bs,
  decay_rate = 0.005) 
opt = Adam(learning_rate=inversetime_decay)
vit16_model = pre_trained_model(input_shape=(384,384,3), n_classes=28, optimizer=opt)

347502902/347502902 [==============================] - 3s 0us/step


In [ ]:
#new test chaning image size
# ModelCheckpoint callback - save best weights #time: 5h1min31s
estopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Mestrado/Experimentos/exp-ktd/exp-ktd-pynb/model_VITb16_pt.weights.best.hdf5', save_best_only=True, verbose=0)
history = vit16_model.fit(x_train1, validation_data= x_val1, epochs=100, verbose=1, callbacks=[checkpoint,estopping], shuffle=True) #,batch_size=16, validation_split=0.1, shuffle=True

Epoch 1/100
756/756 [==============================] - 198s 252ms/step - loss: 2.1389 - accuracy: 0.4375 - val_loss: 1.2045 - val_accuracy: 0.7083
Epoch 2/100
756/756 [==============================] - 182s 241ms/step - loss: 1.1806 - accuracy: 0.6786 - val_loss: 0.8456 - val_accuracy: 0.7738
Epoch 3/100
756/756 [==============================] - 183s 242ms/step - loss: 0.9026 - accuracy: 0.7464 - val_loss: 0.6710 - val_accuracy: 0.8333
Epoch 4/100
756/756 [==============================] - 181s 240ms/step - loss: 0.7656 - accuracy: 0.7811 - val_loss: 0.5520 - val_accuracy: 0.8423
Epoch 5/100
756/756 [==============================] - 182s 241ms/step - loss: 0.6584 - accuracy: 0.8062 - val_loss: 0.4822 - val_accuracy: 0.8780
Epoch 6/100
756/756 [==============================] - 187s 247ms/step - loss: 0.5970 - accuracy: 0.8234 - val_loss: 0.4536 - val_accuracy: 0.8631
Epoch 7/100
756/756 [==============================] - 187s 248ms/step - loss: 0.5437 - accuracy: 0.8360 - val_loss: 0

In [ ]:
vit16_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 384, 384, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 24, 24, 768)       590592    
                                                                 
 reshape_4 (Reshape)         (None, 576, 768)          0         
                                                                 
 class_token (ClassToken)    (None, 577, 768)          768       
                                                                 
 Transformer/posembed_input   (None, 577, 768)         443136    
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 577, 768),       7087872   
 (TransformerBlock)           (None, 12, None, None))      

In [ ]:
scores = vit16_model.evaluate(x_test1)
print('\n%s : %.2f%%' % (vit16_model.metrics_names[1], scores[1] * 100))

280/280 [==============================] - 57s 204ms/step - loss: 0.1055 - accuracy: 0.9679

accuracy : 96.79%


#### Transfer Learning - VITb16 - 224X224

In [ ]:
def pre_trained_model(input_shape, n_classes, optimizer, fine_tune=0):
    """
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. 
               0 = all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    classes = utils.get_imagenet_classes()
    vit_model = vit.vit_b16(
        image_size=input_shape[0],
        activation='sigmoid',
        pretrained=True,
        include_top=True,
        pretrained_top=True)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    
    if fine_tune > 0:
        for layer in vit_model.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in vit_model.layers:
            layer.trainable = False
    '''
    # Turn all the layers trainable.

    for layer in densenet_model.layers:
      layer.trainable = True
    '''
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = vit_model.layers[-2].output
    #top_model = Flatten(name="flatten")(top_model)
    #top_model = Dense(4096, activation='relu')(top_model)
    #top_model = Dense(128, activation='relu')(top_model)
    #top_model = Dropout(0.25)(top_model)
    output_layer = Dense(n_classes, activation='softmax', name='my_predictions')(top_model)
    
    # Group the convolutional base and new output into a Model object.
    model = Model(inputs=vit_model.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
tf.random.set_seed(11) #s

### inverse time decay  ###
inversetime_decay = tf.keras.optimizers.schedules.InverseTimeDecay(
  initial_learning_rate = 0.001, #0.01 - opt2 lr=0.001
  decay_steps = 3360/bs,
  decay_rate = 0.005) 
opt = Adam(learning_rate=inversetime_decay)
vit16_model = pre_trained_model(input_shape=(224,224,3), n_classes=28, optimizer=opt)

In [ ]:
# ModelCheckpoint callback - save best weights #1h3min2s
estopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Mestrado/Experimentos/exp-ktd/exp-ktd-pynb/model_VITb16_224pt.weights.best.hdf5', save_best_only=True, verbose=0)
history = vit16_model.fit(x_train1, validation_data= x_val1, epochs=100, verbose=1, callbacks=[checkpoint,estopping], shuffle=True) #,batch_size=16, validation_split=0.1, shuffle=True

Epoch 1/100
756/756 [==============================] - 87s 93ms/step - loss: 2.1257 - accuracy: 0.4461 - val_loss: 1.2247 - val_accuracy: 0.6756
Epoch 2/100
756/756 [==============================] - 59s 78ms/step - loss: 1.1736 - accuracy: 0.6723 - val_loss: 0.8572 - val_accuracy: 0.7649
Epoch 3/100
756/756 [==============================] - 61s 81ms/step - loss: 0.8905 - accuracy: 0.7560 - val_loss: 0.6993 - val_accuracy: 0.8304
Epoch 4/100
756/756 [==============================] - 70s 93ms/step - loss: 0.7640 - accuracy: 0.7794 - val_loss: 0.5858 - val_accuracy: 0.8452
Epoch 5/100
756/756 [==============================] - 61s 81ms/step - loss: 0.6761 - accuracy: 0.7996 - val_loss: 0.5194 - val_accuracy: 0.8750
Epoch 6/100
756/756 [==============================] - 64s 84ms/step - loss: 0.5936 - accuracy: 0.8218 - val_loss: 0.4859 - val_accuracy: 0.8542
Epoch 7/100
756/756 [==============================] - 61s 81ms/step - loss: 0.5444 - accuracy: 0.8323 - val_loss: 0.4426 - val_ac

In [ ]:
vit16_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 14, 14, 768)       590592    
                                                                 
 reshape (Reshape)           (None, 196, 768)          0         
                                                                 
 class_token (ClassToken)    (None, 197, 768)          768       
                                                                 
 Transformer/posembed_input   (None, 197, 768)         151296    
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 197, 768),       7087872   
 (TransformerBlock)           (None, 12, None, None))        

In [ ]:
vit16_model.load_weights('/content/drive/MyDrive/Mestrado/Experimentos/exp-ktd/exp-ktd-pynb/model_VITb16_pt224.weights.best.hdf5')

In [ ]:
scores = vit16_model.evaluate(x_test1)
print('\n%s : %.2f%%' % (vit16_model.metrics_names[1], scores[1] * 100))

280/280 [==============================] - 15s 54ms/step - loss: 0.2439 - accuracy: 0.9107

accuracy : 91.07%


#### Transfer learning - VITb32 - 384x384

In [ ]:
def pre_trained_model(input_shape, n_classes, optimizer, fine_tune=0):
    """
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training.
    fine_tune: int - The number of pre-trained layers to unfreeze. 
               0 = all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    classes = utils.get_imagenet_classes()
    vit_model = vit.vit_b32(
        image_size=input_shape[0],
        activation='sigmoid',
        pretrained=True,
        include_top=True,
        pretrained_top=True)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    
    if fine_tune > 0:
        for layer in vit_model.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in vit_model.layers:
            layer.trainable = False
    '''
    # Turn all the layers trainable.

    for layer in densenet_model.layers:
      layer.trainable = True
    '''
    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = vit_model.layers[-2].output
    #top_model = Flatten(name="flatten")(top_model)
    #top_model = Dense(4096, activation='relu')(top_model)
    #top_model = Dense(128, activation='relu')(top_model)
    #top_model = Dropout(0.25)(top_model)
    output_layer = Dense(n_classes, activation='softmax', name='my_predictions')(top_model)
    
    # Group the convolutional base and new output into a Model object.
    model = Model(inputs=vit_model.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
tf.random.set_seed(11) #s

### inverse time decay  ###
inversetime_decay = tf.keras.optimizers.schedules.InverseTimeDecay(
  initial_learning_rate = 0.001, #0.01 - opt2 lr=0.001
  decay_steps = 3360/bs,
  decay_rate = 0.005)
#opt = Adam(learning_rate=inversetime_decay) 
opt = Adam(learning_rate=inversetime_decay)
vitb32_model = pre_trained_model(input_shape=(384,384,3), n_classes=28, optimizer=opt)

In [ ]:
# ModelCheckpoint callback - save best weights #1h13min57s
estopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Mestrado/Experimentos/exp-ktd/exp-ktd-pynb/model_VITb32_pt.weights.best.hdf5', save_best_only=True, verbose=0)
history = vitb32_model.fit(x_train1, validation_data= x_val1, epochs=100, verbose=1, callbacks=[checkpoint,estopping], shuffle=True) #,batch_size=16, validation_split=0.1, shuffle=True

Epoch 1/100
756/756 [==============================] - 66s 77ms/step - loss: 2.5748 - accuracy: 0.3016 - val_loss: 1.7449 - val_accuracy: 0.4762
Epoch 2/100
756/756 [==============================] - 48s 63ms/step - loss: 1.6668 - accuracy: 0.5351 - val_loss: 1.3047 - val_accuracy: 0.6161
Epoch 3/100
756/756 [==============================] - 48s 64ms/step - loss: 1.3617 - accuracy: 0.5989 - val_loss: 1.1103 - val_accuracy: 0.6786
Epoch 4/100
756/756 [==============================] - 47s 62ms/step - loss: 1.1829 - accuracy: 0.6508 - val_loss: 0.9501 - val_accuracy: 0.7321
Epoch 5/100
756/756 [==============================] - 48s 63ms/step - loss: 1.0644 - accuracy: 0.6776 - val_loss: 0.8851 - val_accuracy: 0.7232
Epoch 6/100
756/756 [==============================] - 47s 62ms/step - loss: 0.9833 - accuracy: 0.6938 - val_loss: 0.8220 - val_accuracy: 0.7292
Epoch 7/100
756/756 [==============================] - 47s 62ms/step - loss: 0.9166 - accuracy: 0.7136 - val_loss: 0.7574 - val_ac

In [ ]:
vitb32_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 384, 384, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 12, 12, 768)       2360064   
                                                                 
 reshape_3 (Reshape)         (None, 144, 768)          0         
                                                                 
 class_token (ClassToken)    (None, 145, 768)          768       
                                                                 
 Transformer/posembed_input   (None, 145, 768)         111360    
 (AddPositionEmbs)                                               
                                                                 
 Transformer/encoderblock_0   ((None, 145, 768),       7087872   
 (TransformerBlock)           (None, 12, None, None))      

In [ ]:
vitb32_model.load_weights('/content/drive/MyDrive/Mestrado/Experimentos/exp-ktd/exp-ktd-pynb/model_VITb32_pt.weights.best.hdf5')

In [ ]:
scores = vitb32_model.evaluate(x_test1)
print('\n%s : %.2f%%' % (vitb32_model.metrics_names[1], scores[1] * 100))

280/280 [==============================] - 13s 46ms/step - loss: 0.3392 - accuracy: 0.8786

accuracy : 87.86%
